<div align="center">

# **DELIVERY 2**
## **Indexing and Evaluation**

</div>

---

## **PART 1: Indexing**

### **STEP 1 — Build inverted index:**


### **Main Code**

In [1]:
import json
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Set, Any, Iterable
from pathlib import Path
import sys

NOTEBOOK_DIR = Path().resolve()
REPO_ROOT = NOTEBOOK_DIR.parents[1] if NOTEBOOK_DIR.name in {"part_1", "part_2"} else NOTEBOOK_DIR
sys.path.append(str(REPO_ROOT / "project_progress"))
from utils.preprocessing import preprocess_text_field



# Path
NOTEBOOK_DIR = Path().resolve()
REPO_ROOT = NOTEBOOK_DIR.parents[1]          
DATA_DIR = REPO_ROOT / "data"
INPUT = DATA_DIR / "fashion_products_dataset_enriched.json"

INDEX_DIR = DATA_DIR / "index"
INDEX_DIR.mkdir(parents=True, exist_ok=True)

INDEX_FILE = INDEX_DIR / "boolean_inverted_index.json"
DOCMAP_FILE = INDEX_DIR / "docid_pid_map.json"
FIELDS_FILE = INDEX_DIR / "indexed_fields.json"

print(f"Reading enriched dataset: {INPUT}")
print(f"Index will be saved in:   {INDEX_DIR}")


if not INPUT.exists():
    raise FileNotFoundError(f"Enriched dataset not found: {INPUT}")
docs: List[Dict[str, Any]] = json.loads(INPUT.read_text(encoding="utf-8"))
print(f"Loaded {len(docs)} docs")


INDEXED_TEXT_FIELDS = [
    "title_clean",
    "description_clean",
    "metadata_clean",   
]


# doc_id is an integer, stable order = index in list
docid_to_pid: Dict[int, str] = {}
pid_to_docid: Dict[str, int] = {}

for i, r in enumerate(docs):
    pid = r.get("pid")
    if not pid:
        pid = r.get("_id", f"missing_pid_{i}")
    docid_to_pid[i] = pid
    pid_to_docid[pid] = i

def _doc_tokens(record: Dict[str, Any], fields: Iterable[str]) -> List[str]:
    toks: List[str] = []
    for f in fields:
        val = record.get(f)
        if not val:
            continue
        # We already have cleaned strings; just split.
        toks.extend(str(val).split())
    return toks


# Build inverted index 
vocab: Dict[str, Set[int]] = defaultdict(set)

for doc_id, rec in enumerate(docs):
    tokens = _doc_tokens(rec, INDEXED_TEXT_FIELDS)
    if not tokens:
        continue
    # Use unique terms per doc for Boolean presence posting
    for term in set(tokens):
        vocab[term].add(doc_id)

# Convert sets to sorted lists for compactness and efficient AND intersections
inverted_index: Dict[str, List[int]] = {t: sorted(list(s)) for t, s in vocab.items()}
print(f"Vocabulary size: {len(inverted_index):,}")


INDEX_FILE.write_text(json.dumps(inverted_index), encoding="utf-8")
DOCMAP_FILE.write_text(json.dumps({"docid_to_pid": docid_to_pid}, ensure_ascii=False), encoding="utf-8")
FIELDS_FILE.write_text(json.dumps({"indexed_fields": INDEXED_TEXT_FIELDS}, ensure_ascii=False, indent=2), encoding="utf-8")

print(f"Saved inverted index to: {INDEX_FILE}")
print(f"Saved doc map         to: {DOCMAP_FILE}")
print(f"Saved fields          to: {FIELDS_FILE}")


REQUIRED_OUTPUT_FIELDS = [
    "pid", "title", "description", "brand", "category", "sub_category",
    "product_details", "seller", "out_of_stock", "selling_price", "discount",
    "actual_price", "average_rating", "url"
]

def _query_tokens(q: str) -> List[str]:
    # Use the same normalization and stemming pipeline as Step 1
    proc = preprocess_text_field(q or "")
    return proc["tokens"]

def _intersect_sorted(a: List[int], b: List[int]) -> List[int]:
    """Intersect two sorted posting lists."""
    i=j=0
    out: List[int] = []
    while i < len(a) and j < len(b):
        if a[i] == b[j]:
            out.append(a[i])
            i+=1; j+=1
        elif a[i] < b[j]:
            i+=1
        else:
            j+=1
    return out

def search_and(query: str, fields: List[str] = None, k: int = 20) -> List[Dict[str, Any]]:
    """
    Conjunctive (AND) Boolean search.
    Every returned doc must contain ALL query terms (after preprocessing).
    Returns up to k full records with the required fields (when present).
    """
    _ = fields  # kept for future extension; current index already built over INDEXED_TEXT_FIELDS
    q_terms = _query_tokens(query)
    if not q_terms:
        return []

    # Load postings lists; if any term not in vocab -> empty result
    postings_lists: List[List[int]] = []
    for t in q_terms:
        p = inverted_index.get(t)
        if not p:
            return []
        postings_lists.append(p)

    # Intersect from shortest to longest for speed
    postings_lists.sort(key=len)
    result_ids = postings_lists[0]
    for pl in postings_lists[1:]:
        result_ids = _intersect_sorted(result_ids, pl)
        if not result_ids:
            break

    # Map to records and keep only required output fields (when present)
    out: List[Dict[str, Any]] = []
    for did in result_ids[:k]:
        rec = docs[did]
        # Build a thin view with required fields (include only those present)
        view = {f: rec.get(f) for f in REQUIRED_OUTPUT_FIELDS if f in rec}
        # Always include pid
        if "pid" not in view:
            view["pid"] = rec.get("pid") or docid_to_pid.get(did)
        out.append(view)
    return out

Reading enriched dataset: /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/fashion_products_dataset_enriched.json
Index will be saved in:   /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/index
Loaded 28080 docs
Vocabulary size: 9,048
Saved inverted index to: /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/index/boolean_inverted_index.json
Saved doc map         to: /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/index/docid_pid_map.json
Saved fields          to: /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/index/indexed_fields.json


### **STEP 2 — Propose test queries:**

In [32]:
import random

def df(term: str) -> int:
    """Document frequency of a term (0 if absent)."""
    return len(inverted_index.get(term, []))

def in_vocab(token: str) -> bool:
    return token in inverted_index

def stem_phrase(phrase: str) -> list[str]:
    return preprocess_text_field(phrase)["tokens"]

def phrase_ok(phrase: str) -> bool:
    """All tokens exist in vocab AND the AND-query returns at least one result."""
    toks = stem_phrase(phrase)
    if not toks or not all(in_vocab(t) for t in toks):
        return False
    return len(search_and(phrase, k=1)) > 0

def term_popularity_score(tokens: list[str]) -> int:
    """Sum of dfs for quick 'popularity' proxy."""
    return sum(df(t) for t in tokens)

# Candidate lexicons (raw phrases)
gender_phrases  = ["men", "women"]
type_phrases    = [
    "jeans", "shirt", "t shirt", "hoodie", "sweatshirt",
    "track pants", "kurta", "dress", "jacket"
]
color_phrases   = ["black", "blue", "white", "grey", "red", "green", "pink"]
material_phrases= ["cotton", "polyester", "denim", "linen", "silk"]
fit_phrases     = ["slim", "skinny", "regular", "straight", "high waist"]
style_phrases   = ["printed", "solid", "striped", "floral"]

# Keep only phrases whose tokens exist in vocab (post-stemming)
def filter_vocab(phrases):
    good = []
    for p in phrases:
        toks = stem_phrase(p)
        if toks and all(in_vocab(t) for t in toks):
            good.append((p, toks, term_popularity_score(toks)))
    # Sort by popularity descending (PRP)
    return sorted(good, key=lambda x: x[2], reverse=True)

gender_ok   = filter_vocab(gender_phrases)
types_ok    = filter_vocab(type_phrases)
colors_ok   = filter_vocab(color_phrases)
materials_ok= filter_vocab(material_phrases)
fits_ok     = filter_vocab(fit_phrases)
styles_ok   = filter_vocab(style_phrases)

# Compose candidate query templates: (gender) + (type) + (attribute sets)
templates = [
    ["{g}", "{m}", "{t}", "{c}"],                 # gender + material + type + color
    ["{g}", "{t}", "{f}", "{c}"],                 # gender + type + fit + color
    ["{g}", "full sleeve", "{t}", "{m}"],         # gender + sleeve attr + type + material
    ["{g}", "{t}", "{s}", "{c}"],                 # gender + type + style + color
    ["{g}", "high waist", "{t}", "{c}"],          # gender + high waist + type + color
]

def pick(pop_list, k=1):
    return [p[0] for p in pop_list[:k]] if pop_list else []

# Generate diverse, popular queries that actually return hits
proposed = []
attempts = 0
seen_main_types = set()

while len(proposed) < 5 and attempts < 200:
    attempts += 1
    tpl = random.choice(templates)

    g = pick(gender_ok, 1) or ["women"]
    t = pick(types_ok, 1) or ["jeans"]
    m = pick(materials_ok, 1) or ["cotton"]
    c = pick(colors_ok, 1) or ["blue"]
    f = pick(fits_ok, 1) or ["slim"]
    s = pick(styles_ok, 1) or ["printed"]

    phrase = " ".join(
        x.format(g=g[0], t=t[0], m=m[0], c=c[0], f=f[0], s=s[0])
        for x in tpl
    )
    phrase = " ".join(phrase.split())  # clean double spaces

    # Require the AND query to return hits and encourage diversity by not repeating the same main type too much.
    main_type = t[0]
    if phrase_ok(phrase):
        if sum(1 for q in proposed if main_type in q) < 2:
            proposed.append(phrase)

# Fallbacks (just in case)
fallbacks = [
    "women cotton kurta straight",
    "men slim fit formal shirt",
    "women high waist blue jeans",
    "men running shoes black",
    "women printed dress floral"
]
for fb in fallbacks:
    if len(proposed) >= 5:
        break
    if fb not in proposed and phrase_ok(fb):
        proposed.append(fb)

# Deduplicate and trim to 5
seen = set()
unique_proposed = []
for q in proposed:
    if q not in seen:
        unique_proposed.append(q)
        seen.add(q)
proposed = unique_proposed[:5]

print("=== Proposed Test Queries (data-driven) ===")
for i, q in enumerate(proposed, 1):
    hits = len(search_and(q, k=50))
    toks = stem_phrase(q)
    score = term_popularity_score(toks)
    print(f"{i}. {q}  | tokens={toks} | DF-score={score} | matches≈{hits}")

# Store for later evaluation/report
TEST_QUERIES_FILE = DATA_DIR / "index" / "proposed_test_queries.json"
TEST_QUERIES_FILE.write_text(json.dumps({"queries": proposed}, indent=2), encoding="utf-8")
print(f"\nSaved queries to: {TEST_QUERIES_FILE}")

=== Proposed Test Queries (data-driven) ===
1. men shirt solid blue  | tokens=['men', 'shirt', 'solid', 'blue'] | DF-score=50331 | matches≈50
2. men cotton shirt blue  | tokens=['men', 'cotton', 'shirt', 'blue'] | DF-score=58230 | matches≈50
3. women cotton kurta straight  | tokens=['women', 'cotton', 'kurta', 'straight'] | DF-score=35775 | matches≈50
4. men slim fit formal shirt  | tokens=['men', 'slim', 'fit', 'formal', 'shirt'] | DF-score=57466 | matches≈50
5. men running shoes black  | tokens=['men', 'run', 'shoe', 'black'] | DF-score=21967 | matches≈28

Saved queries to: /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/index/proposed_test_queries.json


### **STEP 3 — Ranking our results:**

### **Main Code**

In [37]:
import math
from typing import Tuple

# Configuration
INDEXED_TEXT_FIELDS = ["title_clean", "description_clean", "metadata_clean"]
FIELD_WEIGHTS: Dict[str, float] = {
    "title_clean": 2.0,     
    "description_clean": 1.0,  
    "metadata_clean": 0.7,     
}

# Utilities 
def _tokens_from_fields(record: Dict[str, Any], fields: Iterable[str]) -> Dict[str, Dict[str, int]]:
    """
    Return per-field raw term frequencies:
        { field_name : { term : count_in_that_field } }
    """
    per_field_counts: Dict[str, Dict[str, int]] = {}
    for f in fields:
        txt = record.get(f)
        if not txt:
            continue
        counts: Dict[str, int] = defaultdict(int)
        for t in str(txt).split():
            counts[t] += 1
        if counts:
            per_field_counts[f] = dict(counts)
    return per_field_counts

def _tf_log2(freq: float) -> float:
    """1 + log2(freq) if freq>0 else 0."""
    if freq <= 0:
        return 0.0
    return 1.0 + math.log(freq, 2)

def _idf_log2(df_i: int, N: int) -> float:
    """idf = log2(N / df_i); assumes df_i >= 1."""
    if df_i <= 0 or N <= 0:
        return 0.0
    return math.log(N / df_i, 2)

# Build TF postings & df
N = len(docs)

# term -> { doc_id : (field-weighted raw frequency) }
tf_postings: Dict[str, Dict[int, float]] = defaultdict(dict)
# term -> document frequency
df_counts: Dict[str, int] = defaultdict(int)

for doc_id, rec in enumerate(docs):
    per_field = _tokens_from_fields(rec, INDEXED_TEXT_FIELDS)

    term_freq_weighted: Dict[str, float] = defaultdict(float)
    for field_name, counts in per_field.items():
        w_f = FIELD_WEIGHTS.get(field_name, 1.0)
        for term, f_ij_f in counts.items():
            term_freq_weighted[term] += w_f * f_ij_f

    for term, f_ij in term_freq_weighted.items():
        tf_postings[term][doc_id] = f_ij

# df_i = number of docs where term appears
for term, posting in tf_postings.items():
    df_counts[term] = len(posting)

# Precompute document norms
doc_norms: List[float] = [0.0] * N
for term, posting in tf_postings.items():
    idf_i = _idf_log2(df_counts[term], N)
    if idf_i == 0.0:
        continue
    for d_id, f_ij in posting.items():
        w_dt = _tf_log2(f_ij) * idf_i
        if w_dt != 0.0:
            doc_norms[d_id] += w_dt * w_dt

doc_norms = [math.sqrt(v) if v > 0 else 0.0 for v in doc_norms]

# Ranked search
def search_tfidf(query: str, k: int = 20) -> List[Dict[str, Any]]:
    """
    Rank documents by cosine similarity with TF-IDF (base-2 logs, no smoothing).
    Returns top-k with 'score' plus the required output fields.
    """
    q_proc = preprocess_text_field(query or "")
    q_terms = q_proc["tokens"]
    if not q_terms:
        return []

    # query term frequencies
    q_tf: Dict[str, int] = defaultdict(int)
    for t in q_terms:
        q_tf[t] += 1

    # build query vector
    q_weights: Dict[str, float] = {}
    q_norm_sq = 0.0
    for t, f_q in q_tf.items():
        df_i = df_counts.get(t, 0)
        if df_i <= 0:
            continue  # unseen term
        w_t = _tf_log2(f_q) * _idf_log2(df_i, N)
        if w_t == 0.0:
            continue
        q_weights[t] = w_t
        q_norm_sq += w_t * w_t

    q_norm = math.sqrt(q_norm_sq) if q_norm_sq > 0 else 0.0
    if q_norm == 0.0:
        return []

    # sparse dot product over postings of query terms
    scores: Dict[int, float] = defaultdict(float)
    for t, w_t in q_weights.items():
        posting = tf_postings.get(t)
        if not posting:
            continue
        idf_i = _idf_log2(df_counts[t], N)
        if idf_i == 0.0:
            continue
        for d_id, f_ij in posting.items():
            w_dt = _tf_log2(f_ij) * idf_i
            if w_dt != 0.0:
                scores[d_id] += w_t * w_dt

    # cosine normalization and rank
    ranked: List[Tuple[int, float]] = []
    for d_id, dot in scores.items():
        denom = doc_norms[d_id] * q_norm
        if denom > 0:
            ranked.append((d_id, dot / denom))
    ranked.sort(key=lambda x: x[1], reverse=True)

    results: List[Dict[str, Any]] = []
    for d_id, sc in ranked[:k]:
        rec = docs[d_id]
        view = {f: rec.get(f) for f in REQUIRED_OUTPUT_FIELDS if f in rec}
        if "pid" not in view:
            view["pid"] = rec.get("pid")
        view["score"] = float(sc)
        results.append(view)
    return results

# AND-filtered TF-IDF
def search_tfidf_and(query: str, k: int = 20) -> List[Dict[str, Any]]:
    """
    Conjunctive AND filter first (Boolean), then TF-IDF rank within the survivors.
    Helpful if your teacher wants AND semantics even for ranking.
    """
    # Candidate set via Boolean AND
    cand = search_and(query, k=10_000)
    if not cand:
        return []

    cand_pids = {r["pid"] for r in cand if r.get("pid")}
    cand_ids = {i for i, r in enumerate(docs) if r.get("pid") in cand_pids}

    # Build query vector (same as search_tfidf)
    q_proc = preprocess_text_field(query or "")
    q_terms = q_proc["tokens"]
    if not q_terms:
        return []

    q_tf: Dict[str, int] = defaultdict(int)
    for t in q_terms:
        q_tf[t] += 1

    q_weights: Dict[str, float] = {}
    q_norm_sq = 0.0
    for t, f_q in q_tf.items():
        df_i = df_counts.get(t, 0)
        if df_i <= 0:
            continue
        w_t = _tf_log2(f_q) * _idf_log2(df_i, N)
        if w_t == 0.0:
            continue
        q_weights[t] = w_t
        q_norm_sq += w_t * w_t

    q_norm = math.sqrt(q_norm_sq) if q_norm_sq > 0 else 0.0
    if q_norm == 0.0:
        return []

    scores: Dict[int, float] = defaultdict(float)
    for t, w_t in q_weights.items():
        posting = tf_postings.get(t)
        if not posting:
            continue
        idf_i = _idf_log2(df_counts[t], N)
        if idf_i == 0.0:
            continue
        for d_id, f_ij in posting.items():
            if d_id not in cand_ids:
                continue
            w_dt = _tf_log2(f_ij) * idf_i
            if w_dt != 0.0:
                scores[d_id] += w_t * w_dt

    ranked: List[Tuple[int, float]] = []
    for d_id, dot in scores.items():
        denom = doc_norms[d_id] * q_norm
        if denom > 0:
            ranked.append((d_id, dot / denom))
    ranked.sort(key=lambda x: x[1], reverse=True)

    results: List[Dict[str, Any]] = []
    for d_id, sc in ranked[:k]:
        rec = docs[d_id]
        view = {f: rec.get(f) for f in REQUIRED_OUTPUT_FIELDS if f in rec}
        if "pid" not in view:
            view["pid"] = rec.get("pid")
        view["score"] = float(sc)
        results.append(view)
    return results

# Persist ranked results
def save_ranked_results(out_path: Path, queries: Dict[str, List[str]], use_and_filter: bool = False, k: int = 20) -> Path:
    """
    Save ranked results for groups of queries.
    queries = {"provided": [q1, q2, ...], "proposed": [q3, ...]}
    """
    out = {"provided_queries": {}, "proposed_queries": {}}
    ranker = search_tfidf_and if use_and_filter else search_tfidf

    for group, qlist in queries.items():
        for q in qlist:
            out_key = "provided_queries" if group == "provided" else "proposed_queries"
            out[out_key][q] = ranker(q, k=k)

    out_path.write_text(json.dumps(out, ensure_ascii=False, indent=2), encoding="utf-8")
    return out_path

### **Testing**

In [38]:
# Quick demo on course queries
for q in ["women full sleeve sweatshirt cotton", "men slim jeans blue"]:
    top = search_tfidf(q, k=3)
    print(f"\nTF-IDF (log2) top for: {q!r}")
    for r in top:
        print(f"  {r['score']:.4f} | {r.get('pid')} | {(r.get('title') or '')[:80]}")

# Save ranked results for report/repro
RANKED_OUT = (DATA_DIR / "index" / "ranked_results.json")
queries_for_report = {
    "provided": [
        "women full sleeve sweatshirt cotton",
        "men slim jeans blue",
    ],
    # Optionally load your proposed queries file if it exists
}
pq_file = DATA_DIR / "index" / "proposed_test_queries.json"
if pq_file.exists():
    try:
        queries_for_report["proposed"] = json.loads(pq_file.read_text(encoding="utf-8"))["queries"]
    except Exception:
        pass

out_path = save_ranked_results(RANKED_OUT, queries_for_report, use_and_filter=False, k=20)
print(f"\nRanked results saved to: {out_path}")


TF-IDF (log2) top for: 'women full sleeve sweatshirt cotton'
  0.9074 | SWSFZVTTQCB4SJ7F | Full Sleeve Solid Women Sweatshirt
  0.8760 | SWSFQGS456JAZCQB | Full Sleeve Printed Women Sweatshirt
  0.8724 | SWSFYTYMNTBNARUN | Full Sleeve Solid Women Sweatshirt

TF-IDF (log2) top for: 'men slim jeans blue'
  0.7168 | JEAFSKYHZHSZZC9S | Slim Men Blue Jeans
  0.7147 | JEAFRAQXEKGUPNUN | Slim Men Blue Jeans
  0.7096 | JEAFQF6JBUSEXHVF | Slim Men Blue Jeans

Ranked results saved to: /Users/pauchaves/Documents/Mathematical Engineering in Data Science/4th Year/1st Trimester/WEB RETRIEVAL/irwa-search-engine/data/index/ranked_results.json
